# HST QSO Dataset used by Neeleman and Ribaduo

In [1]:
# imports
from astropy.table import Table

from linetools import utils as ltu

## Summary file

In [2]:
summ_file = os.getenv('RAW_IGMSPEC')+'/HSTQSO/hstqso.lst'

In [5]:
hstqso = Table.read(summ_file, format='ascii')

In [6]:
hstqso[0:3]

QSO_NAME,QSO_ALT_NAME,RA,DEC,QSO_ZEM,INST,GRATE,PID,SPEC_FILE
str10,str24,float64,float64,float64,str4,str17,str19,str23
J0000-1245,PHL2525,0.1017646,-12.76326,0.2,COS,G130M-G160M,12604,J0000-1245_COS.fits
J0001+0709,0,0.4190833,7.1651722,3.234,STIS,G230L,8569,J0001+0709_STIS.fits
J0004-4157,B0002-4214,1.2,-41.96,2.76,FOS,G270H,6577,B0002-4214_FOS-H.fits


## DATE-OBS

In [10]:
date_file = os.getenv('RAW_IGMSPEC')+'/HSTQSO/date_obs_fosh.txt'

In [11]:
date_fosh = Table.read(date_file, format='ascii')

In [12]:
date_fosh[0:3]

SPEC,GRATING,DATE-OBS
str18,str5,str10
B0002-4214_FOS-H,G270H,1996-6-2
B0002+0507_FOS-H,G270H,1993-11-4
B0002+0507_FOS-H,G270H,1993-11-4


## RA/DEC

In [16]:
radec_file = os.getenv('RAW_IGMSPEC')+'/HSTQSO/all_qso_table.txt'

In [17]:
radec = Table.read(radec_file, format='ascii')

In [18]:
radec[0:3]

File_ID,RA,DEC,GL,GB,ZEM,Zlls,NHI,NHI+,NHI-,LLS_TYPE
str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9
B0002-4214,1.20083332,-41.95777893,332.680756,-72.370941,2.76,2.301,17.81,0.0,0.0,"R1,R2"
B0002+0507,1.33416665,5.40311098,102.310738,-55.611916,1.9,0.0,0.0,0.0,0.0,none
B0003+0146,1.448125,2.05061102,100.481804,-58.817158,0.234,0.0,0.0,0.0,0.0,none


In [19]:
fl = [row['File_ID'][0] for row in radec]

### Worry about lenses

In [3]:
q1318 = ltu.radec_to_coord(('13:18:54.67','+29:03:01.49'))

In [5]:
q1318a = ltu.radec_to_coord(('13:18:53.58','+29:03:30.52'))

In [7]:
q1318.separation(q1318a).to('arcsec')

<Angle 32.3576307359541 arcsec>

## Bad spectra

In [9]:
bad1 = '/raid/Raw_IGMspec//HSTQSO/J1330-2056_STIS.fits.gz'

In [10]:
hdu1 = fits.open(bad1)
hdu1.info()

Filename: /raid/Raw_IGMspec//HSTQSO/J1330-2056_STIS.fits.gz
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU       4   ()              
1                BinTableHDU     43   1R x 3C      [1024D, 1024D, 1024D]   


In [ ]:
data = Table